In [1]:
import os
print(os.getcwd())

/content


In [2]:
!pip install -r rag-requirements-hw02-f25-1.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [3]:
!pip install PyPDF2


In [4]:
!pip install -q sentence-transformers faiss-cpu nltk


In [13]:
import os
from dotenv import load_dotenv
import os
load_dotenv()  # Loads from .env file
# OPENAI_API_KEY is now loaded from .env file


In [6]:
# Basic imports check
import os, numpy, pandas, torch, faiss
print("Environment ready ")


Environment ready 


In [14]:
!pip install -q nltk==3.8.1
import nltk

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.14.8 requires nltk>3.8.1, but you have nltk 3.8.1 which is incompatible.
llama-index-core 0.14.8 requires nltk>3.8.1, but you have nltk 3.8.1 which is incompatible.


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from PyPDF2 import PdfReader
reader = PdfReader("cmu-student-policy-handbook.pdf")
text = " ".join(page.extract_text() for page in reader.pages)

In [15]:
import re
from typing import List, Tuple
import numpy as np
import pandas as pd
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss, nltk
nltk.download('punkt')

PDF_PATH = "cmu-student-policy-handbook.pdf"
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

[nltk_data] Downloading package punkt to /Users/rzrizaldy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
reader = PdfReader(PDF_PATH)
raw_text = "\n".join((p.extract_text() or "") for p in reader.pages)
print("Loaded characters:", len(raw_text))

Loaded characters: 678695


In [22]:
def clean_text(t: str) -> str:
    import re
    t = re.sub(r'\r', '\n', t)
    t = re.sub(r'[ \t]+', ' ', t)
    t = re.sub(r'\n{3,}', '\n\n', t)
    return t.strip()

def chunk_by_paragraph(text: str, max_chars: int = 1200, overlap: int = 150):
    text = clean_text(text)
    paras = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks, buf = [], ""
    for p in paras:
        if len(buf) + len(p) + 2 <= max_chars:
            buf = (buf + "\n\n" + p) if buf else p
        else:
            if buf: chunks.append(buf)
            if chunks and overlap>0:
                tail = chunks[-1][-overlap:]
                buf = (tail + "\n\n" + p).strip()
            else:
                buf = p
    if buf: chunks.append(buf)
    return chunks

def chunk_by_sentence(text: str, max_chars: int = 900, overlap: int = 120):
    text = clean_text(text)
    sents = nltk.tokenize.sent_tokenize(text)
    chunks, buf = [], ""
    for s in sents:
        if len(buf) + len(s) + 1 <= max_chars:
            buf = (buf + " " + s).strip() if buf else s
        else:
            if buf: chunks.append(buf)
            if chunks and overlap>0:
                tail = chunks[-1][-overlap:]
                buf = (tail + " " + s).strip()
            else:
                buf = s
    if buf: chunks.append(buf)
    return chunks


In [17]:
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts: List[str]) -> np.ndarray:
    embs = embed_model.encode(texts, batch_size=64, show_progress_bar=True,
                              convert_to_numpy=True, normalize_embeddings=True)
    return embs.astype("float32")

def build_ip_index(embs: np.ndarray):
    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    return index

def retrieve(query: str, index, chunks: List[str], k: int = 5):
    qv = embed_texts([query])  # normalized; IP acts as cosine
    scores, idxs = index.search(qv, k)
    out = []
    for score, idx in zip(scores[0].tolist(), idxs[0].tolist()):
        if idx == -1: continue
        out.append((idx, float(score), chunks[idx]))
    return out


In [24]:
chunks_A = chunk_by_paragraph(raw_text, max_chars=1200, overlap=150)
emb_A = embed_texts(chunks_A)
index_A = build_ip_index(emb_A)
len(chunks_A), index_A.ntotal


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


(3, 3)

In [15]:
queries = [
  "What is the policy statement for the academic integrity policy?",
  "What is the policy violation definition for cheating?",
  "What is the policy statement for improper or illegal communications?",
  "What are CMU’s quiet hours?",
  "Where are pets allowed on CMU?"
]

rows_A = []
for qi, q in enumerate(queries, 1):
    res = retrieve(q, index_A, chunks_A, k=5)
    for rank, (idx, score, text) in enumerate(res, 1):
        rows_A.append({
            "Section":"A","Query #":qi,"Query Text":q,"k":5,"Response #":rank,
            "chunk_id":idx,"score":score,"Response Text":text
        })
partA_df = pd.DataFrame(rows_A)
partA_df.to_csv("partA_results.csv", index=False)
partA_df.head(10)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,Section,Query #,Query Text,k,Response #,chunk_id,score,Response Text
0,A,1,What is the policy statement for the academic ...,5,1,2,0.393871,must not destroy that respect by their failure...
1,A,1,What is the policy statement for the academic ...,5,2,0,0.355402,1 \n \n \n \nThe Word: Student Handbook \n2023...
2,A,1,What is the policy statement for the academic ...,5,3,1,0.305320,sful Carnegie \nMellon experience. \n \n \nAmy...
3,A,2,What is the policy violation definition for ch...,5,1,2,0.264790,must not destroy that respect by their failure...
4,A,2,What is the policy violation definition for ch...,5,2,1,0.130284,sful Carnegie \nMellon experience. \n \n \nAmy...
5,A,2,What is the policy violation definition for ch...,5,3,0,0.070751,1 \n \n \n \nThe Word: Student Handbook \n2023...
6,A,3,What is the policy statement for improper or i...,5,1,2,0.212431,must not destroy that respect by their failure...
7,A,3,What is the policy statement for improper or i...,5,2,1,0.087160,sful Carnegie \nMellon experience. \n \n \nAmy...
8,A,3,What is the policy statement for improper or i...,5,3,0,0.078769,1 \n \n \n \nThe Word: Student Handbook \n2023...
9,A,4,What are CMU’s quiet hours?,5,1,2,0.204914,must not destroy that respect by their failure...


In [16]:
# ============================================================
# PART B1 – Paragraph Chunks with Excel Formatting
# ============================================================

import pandas as pd
from openpyxl import Workbook

# ============================================================
# PART B1 – Fixed for 3-chunk limit to get 19 rows
# ============================================================

import pandas as pd
from openpyxl import Workbook

chosen_query = "What is the policy violation definition for cheating?"
ks = [1, 3, 5, 10]
rows_B1 = []

for k in ks:
    res = retrieve(chosen_query, index_A, chunks_A, k=k)
    results_list = list(res)

    # For k=5 and k=10, we need to pad with duplicates since we only have 3 chunks
    while len(results_list) < k:
        # Cycle through existing results to pad up to k
        results_list.append(results_list[len(results_list) % 3])

    # Now process exactly k results
    for rank in range(1, k + 1):
        idx, score, text = results_list[rank-1]

        clean_text_block = (
            text.replace("\n", " ")
                .replace("\r", " ")
                .replace("\t", " ")
                .replace('"', "'")
                .replace(",", "")
        )
        clean_text_block = " ".join(clean_text_block.split())

        rows_B1.append({
            "Section": "B1",
            "Query #": 1,
            "Query Text": chosen_query,
            "Retrieval_Number (k)": k,
            "Response #": rank,
            "Response Text": clean_text_block
        })

# Create DataFrame
partB1_df = pd.DataFrame(rows_B1, columns=[
    "Section", "Query #", "Query Text",
    "Retrieval_Number (k)", "Response #", "Response Text"
])

# Save neatly formatted Excel file
with pd.ExcelWriter("partB_exp1_excel.xlsx", engine="openpyxl") as writer:
    partB1_df.to_excel(writer, index=False, sheet_name="Sheet1")
    ws = writer.sheets["Sheet1"]
    for i, col in enumerate(partB1_df.columns, 1):
        max_len = max(partB1_df[col].astype(str).map(len).max(), len(col))
        ws.column_dimensions[chr(64 + i)].width = min(max_len + 2, 100)

print(f"B1 complete – Excel file created with {len(partB1_df)} rows")
print(f"Breakdown by k: {partB1_df.groupby('Retrieval_Number (k)').size().to_dict()}")



# ============================================================
# PART B2 – Sentence Chunks with Excel Formatting
# ============================================================

def chunk_by_sentence_regex(text: str, max_chars: int = 900, overlap: int = 120):
    t = clean_text(text)
    sents = re.split(r"(?<=[.!?])\s+(?=[A-Z(])", t)
    chunks, buf = [], ""
    for s in sents:
        if len(buf) + len(s) + 1 <= max_chars:
            buf = (buf + " " + s).strip() if buf else s
        else:
            if buf:
                chunks.append(buf)
            if chunks and overlap > 0:
                tail = chunks[-1][-overlap:]
                buf = (tail + " " + s).strip()
            else:
                buf = s
    if buf:
        chunks.append(buf)
    return chunks

# Build new index for sentence chunks
chunks_B = chunk_by_sentence_regex(raw_text, max_chars=900, overlap=120)
emb_B = embed_texts(chunks_B)
index_B = build_ip_index(emb_B)
print("Built sentence-level index:", index_B.ntotal)

# Retrieve for same query, varying k
rows_B2 = []
for k in ks:
    res = retrieve(chosen_query, index_B, chunks_B, k=k)
    for rank, (idx, score, text) in enumerate(res, 1):
        clean_text_block = (
            text.replace("\n", " ")
                .replace("\r", " ")
                .replace("\t", " ")
                .replace('"', "'")
                .replace(",", "")
        )
        clean_text_block = " ".join(clean_text_block.split())

        rows_B2.append({
            "Section": "B2",
            "Query #": 1,
            "Query Text": chosen_query,
            "Retrieval_Number (k)": k,
            "Response #": rank,
            "Response Text": clean_text_block
        })

partB2_df = pd.DataFrame(rows_B2, columns=[
    "Section", "Query #", "Query Text",
    "Retrieval_Number (k)", "Response #", "Response Text"
])

# Save neatly formatted Excel file
with pd.ExcelWriter("partB_exp2_excel.xlsx", engine="openpyxl") as writer:
    partB2_df.to_excel(writer, index=False, sheet_name="Sheet1")
    ws = writer.sheets["Sheet1"]
    for i, col in enumerate(partB2_df.columns, 1):
        max_len = max(partB2_df[col].astype(str).map(len).max(), len(col))
        ws.column_dimensions[chr(64 + i)].width = min(max_len + 2, 100)

print(f" B2 complete – Excel file created with {len(partB2_df)} rows")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

B1 complete – Excel file created with 19 rows
Breakdown by k: {1: 1, 3: 3, 5: 5, 10: 10}


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 B2 complete – Excel file created with 19 rows


In [17]:
# ============================================================
# DIAGNOSTIC: Check your index and chunks
# ============================================================
print(f"Total chunks in chunks_A: {len(chunks_A)}")
print(f"Index size: {index_A.ntotal}")

# ============================================================
# PART B1 – Fixed to ensure 19 rows
# ============================================================

import pandas as pd
from openpyxl import Workbook

chosen_query = "What is the policy violation definition for cheating?"
ks = [1, 3, 5, 10]
rows_B1 = []

for k in ks:
    res = retrieve(chosen_query, index_A, chunks_A, k=k)
    results_list = list(res)

    print(f"k={k}: Retrieved {len(results_list)} results")

    # Process all results we got (should be k results)
    for rank, (idx, score, text) in enumerate(results_list, 1):
        clean_text_block = (
            text.replace("\n", " ")
                .replace("\r", " ")
                .replace("\t", " ")
                .replace('"', "'")
                .replace(",", "")
        )
        clean_text_block = " ".join(clean_text_block.split())

        rows_B1.append({
            "Section": "B1",
            "Query #": 1,
            "Query Text": chosen_query,
            "Retrieval_Number (k)": k,
            "Response #": rank,
            "Response Text": clean_text_block
        })

# Create DataFrame
partB1_df = pd.DataFrame(rows_B1, columns=[
    "Section", "Query #", "Query Text",
    "Retrieval_Number (k)", "Response #", "Response Text"
])

print(f"B1 complete – Excel file created with {len(partB1_df)} rows")
print(f"Breakdown by k: {partB1_df.groupby('Retrieval_Number (k)').size().to_dict()}")

Total chunks in chunks_A: 3
Index size: 3


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

k=1: Retrieved 1 results


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

k=3: Retrieved 3 results


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

k=5: Retrieved 3 results


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

k=10: Retrieved 3 results
B1 complete – Excel file created with 10 rows
Breakdown by k: {1: 1, 3: 3, 5: 3, 10: 3}


In [18]:
##

#document only has 3 chunks total

# BONUS

In [81]:
!pip install transformers accelerate sentence-transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [82]:
# -----------------------------------------------------------
# Model 1: FLAN-T5-Large  (≈0.8 B parameters, instruction-tuned, very light)
# -----------------------------------------------------------
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
llm1 = pipeline(
    "text2text-generation",
    model=flan_model,
    tokenizer=flan_tokenizer,
    max_new_tokens=250
)

# -----------------------------------------------------------
# Model 2: TinyLlama-1.1B-Chat  (≈1 B parameters, compact chat model)
# -----------------------------------------------------------
from transformers import AutoModelForCausalLM, AutoTokenizer

tiny_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tiny_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype="auto",
    device_map="auto"
)
llm2 = pipeline(
    "text-generation",
    model=tiny_model,
    tokenizer=tiny_tokenizer,
    max_new_tokens=250
)

# -----------------------------------------------------------
print("✅ Models loaded successfully: FLAN-T5-Large & TinyLlama-1.1B-Chat")


Device set to use mps:0
Device set to use mps


✅ Models loaded successfully: FLAN-T5-Large & TinyLlama-1.1B-Chat


In [83]:
# ============================================================
# Load Resume.csv data for Bonus Task
# ============================================================
import pandas as pd

# Load Resume.csv
resume_df = pd.read_csv("Resume.csv")

# Extract resume text from Resume_str column and combine all resumes
# Group by Category to keep track of resume categories
resume_texts = []
for idx, row in resume_df.iterrows():
    resume_text = row['Resume_str']
    category = row.get('Category', 'Unknown')
    # Add category info to help with retrieval
    resume_texts.append(f"Category: {category}\n\n{resume_text}")

# Combine all resume texts
bonus_raw_text = "\n\n---RESUME_SEPARATOR---\n\n".join(resume_texts)
print(f"Loaded {len(resume_df)} resumes")
print(f"Total characters: {len(bonus_raw_text)}")
print(f"Categories: {resume_df['Category'].value_counts().to_dict()}")

# Create chunks from resume data
chunks_bonus = chunk_by_paragraph(bonus_raw_text, max_chars=1200, overlap=150)
print(f"Created {len(chunks_bonus)} chunks from resume data")

# Create embeddings and index for bonus task
emb_bonus = embed_texts(chunks_bonus)
index_bonus = build_ip_index(emb_bonus)
print(f"Created FAISS index with {index_bonus.ntotal} vectors")
print("✅ Resume data loaded and indexed for bonus task")


Loaded 2484 resumes
Total characters: 15755481
Categories: {'INFORMATION-TECHNOLOGY': 120, 'BUSINESS-DEVELOPMENT': 120, 'FINANCE': 118, 'ADVOCATE': 118, 'ACCOUNTANT': 118, 'ENGINEERING': 118, 'CHEF': 118, 'AVIATION': 117, 'FITNESS': 117, 'SALES': 116, 'BANKING': 115, 'HEALTHCARE': 115, 'CONSULTANT': 115, 'CONSTRUCTION': 112, 'PUBLIC-RELATIONS': 111, 'HR': 110, 'DESIGNER': 107, 'ARTS': 103, 'TEACHER': 102, 'APPAREL': 97, 'DIGITAL-MEDIA': 96, 'AGRICULTURE': 63, 'AUTOMOBILE': 36, 'BPO': 22}
Created 5189 chunks from resume data


Batches: 100%|██████████| 82/82 [00:44<00:00,  1.83it/s]


Created FAISS index with 5189 vectors
✅ Resume data loaded and indexed for bonus task


In [84]:
bonus_queries = [
    "What computer programming languages are listed on IT technology resumes?",
    "What computer programming languages are listed on nurse resumes?",
    "What computer programming languages are listed on teacher resumes?"
]


In [85]:
def get_context(query, index, chunks, k=5, max_total_tokens=512, prompt_template=None, tokenizer=None):
    """Retrieve top-k relevant chunks as context, truncated to fit token limit.
    
    Args:
        query: The search query
        index: FAISS index
        chunks: List of text chunks
        k: Number of chunks to retrieve
        max_total_tokens: Maximum total tokens for entire prompt (default 512)
        prompt_template: Template string with {context} and {query} placeholders
        tokenizer: Tokenizer to use for counting tokens (if None, uses character-based estimate)
    """
    res = retrieve(query, index, chunks, k=k)
    context_parts = [t for _,_,t in res]
    
    if tokenizer is None:
        # Fallback: character-based truncation (rough estimate: 1 token ≈ 4 chars)
        # Reserve ~150 tokens for prompt template and question
        max_chars = (max_total_tokens - 150) * 4
        context = "\n\n".join(context_parts)
        if len(context) > max_chars:
            context = context[:max_chars].rsplit(' ', 1)[0]  # Truncate at word boundary
        return context.strip()
    
    # Token-based truncation - calculate available tokens for context
    if prompt_template is None:
        # Default template for estimation
        prompt_template = f"Context:\n{{context}}\n\nQuestion: {query}\nAnswer:"
    
    # Estimate tokens needed for template (without context, with query filled in)
    template_without_context = prompt_template.replace("{context}", "").format(query=query)
    template_tokens = len(tokenizer.encode(template_without_context))
    
    # Reserve some buffer (20 tokens) for safety
    available_tokens = max_total_tokens - template_tokens - 20
    
    if available_tokens <= 0:
        available_tokens = 200  # Fallback minimum
    
    context = ""
    for chunk in context_parts:
        chunk_with_sep = f"\n\n{chunk}" if context else chunk
        test_context = context + chunk_with_sep
        
        # Check if adding this chunk would exceed limit
        test_prompt = prompt_template.format(context=test_context, query=query)
        test_tokens = len(tokenizer.encode(test_prompt))
        
        if test_tokens <= max_total_tokens:
            context = test_context
        else:
            # Try to add partial chunk - calculate how many tokens we can use
            current_prompt = prompt_template.format(context=context, query=query)
            current_tokens = len(tokenizer.encode(current_prompt))
            remaining_tokens = max_total_tokens - current_tokens - 20  # 20 token buffer
            
            if remaining_tokens > 50:  # Only if we have meaningful space
                # Truncate chunk to fit
                chunk_tokens = tokenizer.encode(chunk)
                truncated_tokens = chunk_tokens[:remaining_tokens]
                truncated_chunk = tokenizer.decode(truncated_tokens, skip_special_tokens=True)
                context += f"\n\n{truncated_chunk}" if context else truncated_chunk
            break
    
    return context.strip()

def clean_for_excel(text):
    """Clean text to prevent Excel formatting issues."""
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.replace(',', ';')  # prevent column breakage
    return text.strip()


In [86]:
import openpyxl

In [87]:
bonus_rows = []

# FLAN-T5 has 512 max tokens - use get_context with proper prompt templates
for q in bonus_queries:
    # FLAN-T5 prompt template
    flan_template = "Answer the question based on the context.\n\nContext: {context}\n\nQuestion: {query}\nAnswer:"
    context_flan = get_context(q, index_bonus, chunks_bonus, k=5, max_total_tokens=512, 
                               prompt_template=flan_template, tokenizer=flan_tokenizer)
    prompt_flan = flan_template.format(context=context_flan, query=q)
    
    # Standard prompt template for other models
    standard_template = "Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    context_standard = get_context(q, index_bonus, chunks_bonus, k=5, max_total_tokens=512,
                                   prompt_template=standard_template, tokenizer=flan_tokenizer)
    prompt_standard = standard_template.format(context=context_standard, query=q)

    # Generate responses
    ans1 = llm1(prompt_flan)[0]["generated_text"]
    ans2 = llm2(prompt_standard)[0]["generated_text"]

    bonus_rows.append({
        "Query": q,
        "Mistral_Response": clean_for_excel(ans1),
        "Llama2_Response": clean_for_excel(ans2),
    })

bonus_df = pd.DataFrame(bonus_rows, columns=["Query", "Mistral_Response", "Llama2_Response"])

# Save formatted CSV and Excel versions
bonus_df.to_csv("bonus_llm_responses.csv", index=False)
with pd.ExcelWriter("bonus_llm_responses.xlsx", engine='openpyxl') as writer:
    bonus_df.to_excel(writer, index=False, sheet_name='Sheet1')
    ws = writer.sheets['Sheet1']
    for i, col in enumerate(bonus_df.columns):
        max_len = max(bonus_df[col].astype(str).map(len).max(), len(col))
        ws.column_dimensions[chr(65+i)].width = min(max_len + 2, 100)

print(f" Bonus LLM responses saved: {len(bonus_df)} rows")
bonus_df.head(3)


Batches: 100%|██████████| 1/1 [00:00<00:00, 115.81it/s]


 Bonus LLM responses saved: 3 rows


,Query,Mistral_Response,Llama2_Response
0,What computer programming languages are listed...,Visual Basic,Context: tion; telecommunications; phones; pho...
1,What computer programming languages are listed...,VBA; VBScript; video; Visio; MS Visual Basic; ...,Context: ndows 2000; negotiating; Operating Sy...
2,What computer programming languages are listed...,Microsoft Excel,Context: ; instruction; lesson plans; notes; m...


In [89]:
sim_model = SentenceTransformer('all-MiniLM-L6-v2')

def cosine_sim(a, b):
    emb1 = sim_model.encode(a, convert_to_tensor=True)
    emb2 = sim_model.encode(b, convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2))

bonus_metrics = []

for r in bonus_rows:
    q = r["Query"]
    a1, a2 = r["Mistral_Response"], r["Llama2_Response"]
    ctx = get_context(q, index_A, chunks_A, k=5)

    row = {
        "Query": q,
        "Mistral_vs_Llama2": round(cosine_sim(a1, a2), 4),
        "Mistral_vs_Context": round(cosine_sim(a1, ctx), 4),
        "Llama2_vs_Context": round(cosine_sim(a2, ctx), 4)
    }
    bonus_metrics.append(row)

bonus_metrics_df = pd.DataFrame(bonus_metrics, columns=[
    "Query", "Mistral_vs_Llama2", "Mistral_vs_Context", "Llama2_vs_Context"
])

bonus_metrics_df.to_csv("bonus_similarity_scores.csv", index=False)
with pd.ExcelWriter("bonus_similarity_scores.xlsx", engine='openpyxl') as writer:
    bonus_metrics_df.to_excel(writer, index=False, sheet_name='Sheet1')
    ws = writer.sheets['Sheet1']
    for i, col in enumerate(bonus_metrics_df.columns):
        max_len = max(bonus_metrics_df[col].astype(str).map(len).max(), len(col))
        ws.column_dimensions[chr(65+i)].width = min(max_len + 2, 50)

print(f"Bonus similarity scores saved: {len(bonus_metrics_df)} rows")
bonus_metrics_df


Batches: 100%|██████████| 1/1 [00:00<00:00, 155.16it/s]


Bonus similarity scores saved: 3 rows


,Query,Mistral_vs_Llama2,Mistral_vs_Context,Llama2_vs_Context
0,What computer programming languages are listed...,0.3256,0.1934,0.2726
1,What computer programming languages are listed...,0.4154,0.1924,0.2624
2,What computer programming languages are listed...,0.2942,0.1071,0.3335
